In [1]:
import pandas as pd
import numpy as np

In [2]:
columns = ["household_key", "BASKET_ID", "DAY", "PRODUCT_ID", "QUANTITY", "SALES_VALUE"]
transactions = pd.read_csv(
    "../data/project_transactions.csv",
    dtype={"DAY": "Int8", "QUANTITY": "Int16", "PRODUCT_ID": "Int32"},
)

In [20]:
transactions.head(2)

,household_key,BASKET_ID,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC,date
0,1364,26984896261,842930,1,2.19,31742,0.0,1,0.0,0.0,2016-01-01
1,1364,26984896261,897044,1,2.99,31742,-0.4,1,0.0,0.0,2016-01-01


In [3]:
transactions = transactions.assign(
    date=(
        pd.to_datetime("2016", format="%Y")
        + pd.to_timedelta(transactions["DAY"].sub(1).astype(str) + " days")
    )
).drop(["DAY"], axis=1)

In [4]:
columns = ["AGE_DESC", "INCOME_DESC", "household_key", "HH_COMP_DESC"]
demographics = pd.read_csv(
    "../data/hh_demographic.csv",
    dtype={
        "AGE_DESC": "category",
        "INCOME_DESC": "category",
        "HH_COM_DESC": "category",
    },
)

In [5]:
demographics["household_key"].nunique()

801

In [6]:
transactions.head(2)

,household_key,BASKET_ID,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC,date
0,1364,26984896261,842930,1,2.19,31742,0.0,1,0.0,0.0,2016-01-01
1,1364,26984896261,897044,1,2.99,31742,-0.4,1,0.0,0.0,2016-01-01


In [7]:
agg_trans = transactions.groupby(["household_key"])[["SALES_VALUE"]].sum()
agg_trans.head(2)

,SALES_VALUE
household_key,
1,4330.16
2,1954.34


In [8]:
demo_trans = demographics.merge(
    agg_trans, how="left", left_on=["household_key"], right_on=["household_key"]
)

In [9]:
demo_trans.head()

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key,SALES_VALUE
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,4330.16
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,3400.05
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8,5534.97
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13,13190.92
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16,1512.02


In [15]:
demo_trans.pivot_table(
    index="AGE_DESC", columns="HH_COMP_DESC", values="SALES_VALUE", aggfunc="mean"
).style.background_gradient(cmap="Pastel1_r")

C:\Users\zzbinden\AppData\Local\Temp\ipykernel_29312\3118935214.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  demo_trans.pivot_table(


HH_COMP_DESC,1 Adult Kids,2 Adults Kids,2 Adults No Kids,Single Female,Single Male,Unknown
AGE_DESC,,,,,,
19-24,7268.796667,5428.945000,4020.800000,4576.095556,3216.835000,4911.275000
25-34,5512.196875,5753.973514,5638.515833,4807.440588,4909.522381,7356.270000
35-44,6297.737778,6691.772264,6260.412444,6015.192069,4844.192000,4227.691818
45-54,6632.569167,6610.484490,5839.527027,4549.365405,4636.637083,4843.995682
55-64,3064.870000,4695.655000,5752.413684,4816.148462,3922.546250,7973.750000
65+,4040.810000,5536.866667,4614.108571,4059.699412,3871.556000,2879.290000


In [17]:
columns = ["PRODUCT_ID", "DEPARTMENT"]
products = pd.read_csv("../data/product.csv", usecols=columns)

In [18]:
products.head(2)

,PRODUCT_ID,DEPARTMENT
0,25671,GROCERY
1,26081,MISC. TRANS.


In [21]:
transactions.head(2)

,household_key,BASKET_ID,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC,date
0,1364,26984896261,842930,1,2.19,31742,0.0,1,0.0,0.0,2016-01-01
1,1364,26984896261,897044,1,2.99,31742,-0.4,1,0.0,0.0,2016-01-01


In [22]:
demographics.head(2)

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [23]:
prod_trans = transactions.merge(
    products, how="inner", left_on="PRODUCT_ID", right_on="PRODUCT_ID"
)

In [24]:
prod_trans.head(2)

,household_key,BASKET_ID,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC,date,DEPARTMENT
0,1364,26984896261,842930,1,2.19,31742,0.0,1,0.0,0.0,2016-01-01,GROCERY
1,1364,26984896261,897044,1,2.99,31742,-0.4,1,0.0,0.0,2016-01-01,GROCERY


In [25]:
full_df = prod_trans.merge(
    demographics, how="inner", left_on="household_key", right_on="household_key"
)

In [26]:
full_df.head(2)

,household_key,BASKET_ID,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC,date,DEPARTMENT,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC
0,1364,26984896261,842930,1,2.19,31742,0.0,1,0.0,0.0,2016-01-01,GROCERY,65+,B,100-124K,Homeowner,Single Female,1,None/Unknown
1,1364,26984896261,897044,1,2.99,31742,-0.4,1,0.0,0.0,2016-01-01,GROCERY,65+,B,100-124K,Homeowner,Single Female,1,None/Unknown


In [33]:
full_df.pivot_table(
    index="DEPARTMENT", columns="AGE_DESC", values="SALES_VALUE", aggfunc="sum"
).style.background_gradient(cmap="Pastel1_r", axis=0)

C:\Users\zzbinden\AppData\Local\Temp\ipykernel_29312\723400945.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  full_df.pivot_table(


AGE_DESC,19-24,25-34,35-44,45-54,55-64,65+
DEPARTMENT,,,,,,
,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AUTOMOTIVE,11.640000,21.250000,72.580000,55.920000,0.000000,16.370000
CHEF SHOPPE,81.300000,134.160000,348.530000,418.240000,80.860000,149.240000
CNTRL/STORE SUP,2.000000,0.000000,1.000000,9.950000,2.000000,0.100000
COSMETICS,698.630000,2273.030000,4362.020000,5187.570000,986.260000,600.900000
COUP/STR & MFG,7.490000,48.420000,121.200000,154.550000,40.680000,20.490000
DAIRY DELI,3.800000,3.850000,7.390000,16.750000,3.140000,1.940000
DELI,4043.300000,18181.940000,34577.290000,44334.220000,9850.540000,10462.330000
DELI/SNACK BAR,0.000000,0.000000,6.980000,1.560000,0.000000,3.310000
